# Stupid Art Project 2017 - Random Holy Text
To fit the stupid art theme "Religion" and my current focus on machine learning, I'm going to generate some random 

The code below was picked up from this site:
http://agiliq.com/blog/2009/06/generating-pseudo-random-text-with-markov-chains-u/
It'll be my first attempt at random holy text generation.

In [1]:
import random


class Markov(object):
    def __init__(self, open_file):
        self.cache = {}
        self.open_file = open_file
        self.words = self.file_to_words()
        self.word_size = len(self.words)
        self.database()

    def file_to_words(self):
        self.open_file.seek(0)
        data = self.open_file.read()
        words = data.split()
        return words

    def triples(self):
        """ Generates triples from the given data string. So if our string were
                "What a lovely day", we'd generate (What, a, lovely) and then
                (a, lovely, day).
        """

        if len(self.words) < 3:
            return

        for i in range(len(self.words) - 2):
            yield (self.words[i], self.words[i + 1], self.words[i + 2])

    def database(self):
        for w1, w2, w3 in self.triples():
            key = (w1, w2)
            if key in self.cache:
                self.cache[key].append(w3)
            else:
                self.cache[key] = [w3]

    def generate_markov_text(self, size=25):
        seed = random.randint(0, self.word_size - 3)
        seed_word, next_word = self.words[seed], self.words[seed + 1]
        w1, w2 = seed_word, next_word
        gen_words = []
        for i in range(size):
            gen_words.append(w1)
            w1, w2 = w2, random.choice(self.cache[(w1, w2)])
        gen_words.append(w2)

        return ' '.join(gen_words)

I'll use the old testament first. I got the text from project Gutenberg, and have removed the Gutenberg pre and postamble. I suspect the text will be too big to run efficiently in the code above, as it is just straight python. We shall see.

In [2]:
with open('sources/old testament.txt') as bible:
    bible_gen = Markov(bible)

In [3]:
bible_gen.generate_markov_text()

'he will take you twelve men out of the first year without blemish: 29:24 Their meat offering unto one of another. 11:10 And Joshua rose in'

In [4]:
%%timeit
bible_gen.generate_markov_text()

10000 loops, best of 3: 39.6 µs per loop


It works, and it is fast. But the punctuation and the passage numbering are not the best. I'll try post processing the responses.

Lets first treat the passage numbers appearing mid passage, things like '7:12'. 

In [5]:
import re

'7:12' appearing at the beginning of a passage is desirable actually, so I'd liek to keep those, and possibly force those later. It is the other passage numbers appearing in the middle or end of passages that we want to avoid. Those patterns are guaranteed to have a space in front of them, ie ' 7:12', so I can regex to find and replace using that pattern.

In [6]:
passage_num_pattern = re.compile(r' \d+:\d+')

In [7]:
for _ in range(3):
    passage = bible_gen.generate_markov_text()
    print(passage)
    passage = passage_num_pattern.sub('', passage)
    print(passage)
    

very deed, as the Father hath sent me to hear as the LORD said, I know thy abode, and laid him in Jerusalem; and the did,
very deed, as the Father hath sent me to hear as the LORD said, I know thy abode, and laid him in Jerusalem; and the did,
polluted with blood. 106:39 Thus were the sons of Ham, and Japheth. 1:5 The sun and the fire for a possession among us: but ye, ye
polluted with blood. Thus were the sons of Ham, and Japheth. The sun and the fire for a possession among us: but ye, ye
her, Woman, why weepest thou? whom seekest thou? or, Why talkest thou with her? 4:28 The woman saith unto them, saying, Go and sit down, my
her, Woman, why weepest thou? whom seekest thou? or, Why talkest thou with her? The woman saith unto them, saying, Go and sit down, my


But I want to create random passages, so they need to start with a passage number, and contain a few "complete" sentences, ie they end in a period. I think I need to alter the generating method, and I think the alterations will be particular to the old testament. So I'll extend the Markov class with a generating method specialy for this text of the old testament. 

In [8]:
class OldTestaPassagesMarkov(Markov):
    passage_num_pattern = re.compile(r'\d+:\d+')
    
    def generate_markov_text(self, seed_word='', min_words=25):
        
        # Process a user given seed_word
        seed_word_locations = [idx for idx, word in enumerate(self.words) if word.lower() == seed_word.lower()]
            
        if seed_word_locations:
            seed = random.choice(seed_word_locations)
        else:
            print(seed_word + ' is not in Old Testament')
            seed = random.randint(0, self.word_size - 3)
            
        w1, w2 = self.words[seed], self.words[seed + 1]
        gen_words = [w1, w2]
        # go until we have enough words and end in a period
        while w2[-1] != '.' or len(gen_words) < min_words:
            w1, w2 = w2, random.choice(self.cache[(w1, w2)])
            if passage_num_pattern.findall(w2):
                # Avoid adding passage numbers to the middle of the passage.
                # Also end a sentence when a passage number would have gone in.
                new_w1 = w1.replace(':', '.').replace(';', '.')
                gen_words[-1] = new_w1
            else:                
                gen_words.append(w2)

        return ' '.join(gen_words)

In [9]:
with open('sources/old testament.txt') as bible:
    bible_gen = OldTestaPassagesMarkov(bible)

In [10]:
bible_gen.generate_markov_text('12:13', min_words=10)

'12:13 Howbeit there is no death. 13:1 A wise man hearken unto their idols, nor make any cuttings in your land with the wicked? 18:24 Peradventure there shall no more children, tossed to and fro throughout the houses of joy with her, which hath power on her head, and the cities which he made them rejoice that seek their life: and he said unto his daughter came out of the tribes of Israel, even the dregs of the asses, that I and this writing was signed, he went out the work which ye have made them rejoice that seek his soul.'

In [11]:
len('So king Solomon passed all the cities that are entering to go into his own language.')

84

## Randomnly generate the passage chapter and verse numbers. 
The chapter:verse (4:34) pairs are good places to start the passages, as they are always starts of sentences, so I'll always seed the generation process with one of those pairs. But, some chapter and verse combos do not exist in the text, so I can't just randomly generate the pairs. Though, I can just sample directly from the set of pairs from the words list. 

In [12]:
passage_numbers = set()

for word in bible_gen.words:
    found_pattern = bible_gen.passage_num_pattern.findall(word)
    if found_pattern:
        passage_numbers.add(found_pattern[0])
list(passage_numbers)[:10]

['30:16',
 '38:8',
 '43:25',
 '6:32',
 '34:19',
 '40:19',
 '14:24',
 '29:38',
 '23:8',
 '50:33']

## Twitter bot experiment
Following this website, I setup a twitter account (HolyStupidArt) and a twitter app attached to that account.
http://www.dototot.com/how-to-write-a-twitter-bot-with-python-and-tweepy/

I've also setup my api accesses, and have sent my first message, the one about king Soloman above. Now, I'm going to experiment with the twitter interface to see if I can make the both respond to direct messages. 

In [92]:
import tweepy

from twitter_secrets import api_tokens as at


# enter the corresponding information from your Twitter application:
auth = tweepy.OAuthHandler(at['CONSUMER_KEY'], at['CONSUMER_SECRET'])
auth.set_access_token(at['ACCESS_KEY'], at['ACCESS_SECRET'])
api = tweepy.API(auth)

In [93]:
api.update_status('@salvor7 Woman, why weepest thou? whom seekest thou? or, Why talkest thou with her? The woman saith unto them, saying, Go and sit down')

Status(in_reply_to_user_id_str='159166590', is_quote_status=False, favorite_count=0, author=User(friends_count=1, description='', profile_background_image_url=None, translator_type='none', contributors_enabled=False, profile_image_url_https='https://abs.twimg.com/sticky/default_profile_images/default_profile_1_normal.png', url=None, name='HolyStupidArtBot', _api=<tweepy.api.API object at 0x00000213A2B137F0>, location='', has_extended_profile=False, profile_sidebar_fill_color='DDEEF6', listed_count=0, is_translation_enabled=False, profile_image_url='http://abs.twimg.com/sticky/default_profile_images/default_profile_1_normal.png', geo_enabled=False, profile_use_background_image=True, profile_background_color='F5F8FA', utc_offset=None, screen_name='HolyStupidArt', verified=False, created_at=datetime.datetime(2016, 11, 12, 0, 3, 1), statuses_count=2, profile_text_color='333333', _json={'friends_count': 1, 'description': '', 'profile_background_image_url': None, 'translator_type': 'none', '

In [40]:
user = api.followers_ids()

In [42]:
user

[159166590]

In [50]:
api.direct_messages()

[]

In [30]:
dm = api.sent_direct_messages()

In [45]:
import time

In [47]:
followers = set(api.followers_ids())

while True:
    time.sleep(15)
    current_followers = set(api.followers_ids())
    if current_followers != followers:
        for user_id in current_followers - followers:
            api.send_direct_message(user_id=user_id, text='Hi new follower')
            print('DM sent to new follower')
        followers = current_followers

DM sent to new follower


KeyboardInterrupt: 

Below is from the answer here: http://stackoverflow.com/questions/39237420/how-to-thread-a-tweepy-stream

In [57]:

import threading
import time
from sys import stdout

# Only data wihtin a class are actually shared by the threads
class Counter(object):
    counter = 0
    stop = False

# Function that will be executed in parallel with the rest of the code
def function_1():
    for i in range(10):
        if c.stop: return # With more will exit faster
        time.sleep(2)
        c.counter += 1
        if c.stop: return

# Create a class instance
c = Counter()

# Thread function_1
d = threading.Thread(target=function_1)
d.start()

# Exit the thread properly before exiting...
try:
    for j in range(100):
        stdout.write('\r{:}'.format(c.counter))
        stdout.flush()
        time.sleep(1)
except:
    c.stop = True

10

Not sure how to make this work for my example right off. I'll check another solution first.

This one is from http://stackoverflow.com/questions/31116213/streaming-twitter-direct-messages
It works great. The direct messages are going in and out on_data, not on_direct_message; not sure what that is about. 

In [ ]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy import API

from tweepy.streaming import StreamListener


class StdOutListener( StreamListener ):

    def __init__( self ):
        self.tweetCount = 0

    def on_connect( self ):
        print("Connection established!!")

    def on_disconnect( self, notice ):
        print("Connection lost!! : ", notice)

    def on_direct_message( self, status ):
        print("Entered on_direct_message()")
        try:
            print(status, flush = True)
            return True
        except BaseException as e:
            print("Failed on_direct_message()", str(e))

    def on_error( self, status ):
        print(status)

def main():

    try:
        auth = tweepy.OAuthHandler(at['CONSUMER_KEY'], at['CONSUMER_SECRET'])
        auth.secure = True
        auth.set_access_token(at['ACCESS_KEY'], at['ACCESS_SECRET'])

        api = API(auth)

        # If the authentication was successful, you should
        # see the name of the account print out
        print(api.me().name)

        stream = Stream(auth, StdOutListener())

        stream.userstream()

    except BaseException as e:
        print("Error in main()", e)

In [59]:
main()

HolyStupidArtBot
Connection established!!
Entered on_data()
{"friends":[159166590]}


Entered on_data()
{"direct_message":{"id":797525619047211016,"id_str":"797525619047211016","text":"Hi bot","sender":{"id":159166590,"id_str":"159166590","name":"Andrew Brown","screen_name":"salvor7","location":"vancouver","url":null,"description":"Today, who knows","protected":false,"followers_count":111,"friends_count":170,"listed_count":1,"created_at":"Thu Jun 24 17:02:23 +0000 2010","favourites_count":19,"utc_offset":-21600,"time_zone":"Central Time (US & Canada)","geo_enabled":true,"verified":false,"statuses_count":215,"lang":"en","contributors_enabled":false,"is_translator":false,"is_translation_enabled":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_image_url":"http:\/\/pbs.twimg.

In [79]:
import json
from pprint import pprint

In [61]:
s = '{"direct_message":{"id":797525813570568196,"id_str":"797525813570568196","text":"test","sender":{"id":797228167974895616,"id_str":"797228167974895616","name":"HolyStupidArtBot","screen_name":"HolyStupidArt","location":null,"url":null,"description":null,"protected":false,"followers_count":1,"friends_count":1,"listed_count":0,"created_at":"Sat Nov 12 00:03:01 +0000 2016","favourites_count":0,"utc_offset":null,"time_zone":null,"geo_enabled":false,"verified":false,"statuses_count":1,"lang":"en","contributors_enabled":false,"is_translator":false,"is_translation_enabled":false,"profile_background_color":"F5F8FA","profile_background_image_url":null,"profile_background_image_url_https":null,"profile_background_tile":false,"profile_image_url":"http:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","profile_image_url_https":"https:\/\/abs.twimg.com\/sticky\/default_profile_images\/default_profile_1_normal.png","profile_link_color":"1DA1F2","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"default_profile":true,"default_profile_image":true,"following":false,"follow_request_sent":false,"notifications":false,"translator_type":"none"},"sender_id":797228167974895616,"sender_id_str":"797228167974895616","sender_screen_name":"HolyStupidArt","recipient":{"id":159166590,"id_str":"159166590","name":"Andrew Brown","screen_name":"salvor7","location":"vancouver","url":null,"description":"Today, who knows","protected":false,"followers_count":111,"friends_count":170,"listed_count":1,"created_at":"Thu Jun 24 17:02:23 +0000 2010","favourites_count":19,"utc_offset":-21600,"time_zone":"Central Time (US & Canada)","geo_enabled":true,"verified":false,"statuses_count":215,"lang":"en","contributors_enabled":false,"is_translator":false,"is_translation_enabled":false,"profile_background_color":"C0DEED","profile_background_image_url":"http:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_image_url_https":"https:\/\/abs.twimg.com\/images\/themes\/theme1\/bg.png","profile_background_tile":false,"profile_image_url":"http:\/\/pbs.twimg.com\/profile_images\/731256308909772800\/uHAlRwB-_normal.jpg","profile_image_url_https":"https:\/\/pbs.twimg.com\/profile_images\/731256308909772800\/uHAlRwB-_normal.jpg","profile_link_color":"1DA1F2","profile_sidebar_border_color":"C0DEED","profile_sidebar_fill_color":"DDEEF6","profile_text_color":"333333","profile_use_background_image":true,"default_profile":true,"default_profile_image":false,"following":false,"follow_request_sent":false,"notifications":false,"translator_type":"none"},"recipient_id":159166590,"recipient_id_str":"159166590","recipient_screen_name":"salvor7","created_at":"Sat Nov 12 19:45:45 +0000 2016","entities":{"hashtags":[],"symbols":[],"user_mentions":[],"urls":[]}}}'

In [70]:
json_obj = json.loads(s)

In [80]:
pprint(json_obj)

{'direct_message': {'created_at': 'Sat Nov 12 19:45:45 +0000 2016',
                    'entities': {'hashtags': [],
                                 'symbols': [],
                                 'urls': [],
                                 'user_mentions': []},
                    'id': 797525813570568196,
                    'id_str': '797525813570568196',
                    'recipient': {'contributors_enabled': False,
                                  'created_at': 'Thu Jun 24 17:02:23 +0000 '
                                                '2010',
                                  'default_profile': True,
                                  'default_profile_image': False,
                                  'description': 'Today, who knows',
                                  'favourites_count': 19,
                                  'follow_request_sent': False,
                                  'followers_count': 111,
                                  'following': False,
           

In [86]:
json_obj['direct_message']['sender']['name'], json_obj['direct_message']['sender']['id'], json_obj['direct_message']['text']

('HolyStupidArtBot', 797228167974895616, 'test')

In [85]:
json_obj.keys()

dict_keys(['direct_message'])

In [94]:
from tweepy.models import Status

In [95]:
status = Status.parse(api=api, json=json_obj)

In [101]:
dm = status._json['direct_message']

{'created_at': 'Sat Nov 12 19:45:45 +0000 2016',
 'entities': {'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []},
 'id': 797525813570568196,
 'id_str': '797525813570568196',
 'recipient': {'contributors_enabled': False,
  'created_at': 'Thu Jun 24 17:02:23 +0000 2010',
  'default_profile': True,
  'default_profile_image': False,
  'description': 'Today, who knows',
  'favourites_count': 19,
  'follow_request_sent': False,
  'followers_count': 111,
  'following': False,
  'friends_count': 170,
  'geo_enabled': True,
  'id': 159166590,
  'id_str': '159166590',
  'is_translation_enabled': False,
  'is_translator': False,
  'lang': 'en',
  'listed_count': 1,
  'location': 'vancouver',
  'name': 'Andrew Brown',
  'notifications': False,
  'profile_background_color': 'C0DEED',
  'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png',
  'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png',
  'profile_background_til